## TPC-H Snowflake Import
Import data from GCS into Snowflake dataset

In [ ]:
import config, schema, sf, load

### Start Snowflake WAREHOUSE

In [ ]:
# initiate SnowflakeHelper
sf_helper = sf.SnowflakeHelper(config)

# start Warehouse
sf_helper.start_warehouse()

## Setup STAGE: Link GCS data source in Snowflake:

In [ ]:
integration_id = sf_helper.create_integration(table_name, gcs_location)

#### Create NAMED FILE FORMAT

In [ ]:
"""
corresponding query example:

create or replace file format h_1gb_csv_format
  type = csv
  field_delimiter = '|'
  skip_header = 1
  null_if = ('NULL', 'null')
  empty_field_as_null = true
  compression = none;
"""

#### Create STORAGE INTEGRATION

In [ ]:
"""
corresponding query example:

 CREATE STORAGE INTEGRATION h_1gb
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = GCS
  ENABLED = TRUE
  STORAGE_ALLOWED_LOCATIONS = ('gcs://tpc-benchmark-5947/');
"""

#### Grant USAGE privileges on new integration

In [ ]:
"""
corresponding query example:

  grant create stage on schema public to role ACCOUNTADMIN;;
  grant usage on integration h_1gb to role ACCOUNTADMIN;
"""

#### Create Data STAGE

In [ ]:
"""
corresponding query example:

create stage h_1gb_stage
  url = 'gcs://tpc-benchmark-5947/h_1GB_part.tbl'
  storage_integration = h_1gb
  file_format = h_1gb_csv_format;
"""

#### Test STAGE

In [ ]:
"""
corresponding query example:

DESC STORAGE INTEGRATION h_1gb;
list @h_1gb_stage;
"""

### Import Data from STAGE to target table

In [ ]:
sf.import_data(integration_id, table_name)
"""
corresponding query example:

copy into part from @h_1gb_stage;
"""

### Suspend WAREHOUSE

In [ ]:
sf.suspend_warehouse(config.warehouse_name)

"""
corresponding query example:

alter WAREHOUSE test1 suspend;
"""